In [1]:
import numpy as np
import pandas as pd
import copy

In [2]:
train_df = pd.read_excel("训练.xlsx")
test1_df = pd.read_excel("测试A.xlsx")
test2_df = pd.read_excel("测试B.xlsx")

In [3]:
train_y = train_df[train_df.columns[-1]]
train_y.shape

(500,)

In [4]:
train_id = train_df['ID']
test1_id = test1_df['ID']
test2_id = test2_df['ID']
train_df.drop(['ID'], axis=1, inplace=True)
test1_df.drop(['ID'], axis=1, inplace=True)
test2_df.drop(['ID'], axis=1, inplace=True)

In [5]:
new_column = test1_df.columns
train_df = train_df[new_column]

In [6]:
trainx = pd.concat([train_df, test1_df, test2_df], axis=0)
trainx.shape

(721, 8027)

In [14]:
#drop the names first
names = train_df.columns
dts = train_df.dtypes
drop_names = []
ratio = 0.1
for i in range(len(names)):
    n = names[i]
    tp = dts[i]
    trd = train_df[n]
    ted = test1_df[n]
    if 'object' == str(tp):
        continue
    else:
        misstr = np.sum(trd.isnull())/500
        misste = np.sum(ted.isnull())/100
        if np.abs(misstr-misste) > ratio:
            drop_names.append(n)
        else:
            if 'int' in str(tp):
                continue
            trrd = trd[trd.notnull()]
            terd = ted[ted.notnull()]
            m1 = np.mean(trrd)
            s1 = np.std(trrd)
            m2 = np.mean(terd)
            s2 = np.std(terd)
            if np.abs(m1-m2)>ratio*np.abs(m1) and np.abs(s1-s2)>ratio*np.abs(s1):
                drop_names.append(n)
print(len(drop_names))

855


In [15]:
trainx.drop(drop_names, axis=1, inplace=True)
trainx.shape

(721, 7172)

In [16]:
# if column's type is object then pass. else remove anyone that has stddev == 0
dropname = []
onehotname = []
ints = []
dtypes = trainx.dtypes
names = trainx.columns
nonans = []
nans = []
for i in range(len(names)):
    n = names[i]
    d = dtypes[i]
    if str(d) == 'object':
        onehotname.append(n)
    elif 'int' in str(d):
        ints.append(n)
    else:
        c = trainx[n]
        x = np.sum(c.isnull().values)
        if x == 0:
            s = np.std(c.values)
            if s == 0:
                dropname.append(n)
            else:
                nonans.append(n)
        else:
            r = c.notnull()
            rc = c[r]
            s = np.std(rc)
            if x > 500:
                dropname.append(n)
            else:
                if s == 0 :
                    if x > 200:
                        c[c.isnull()] = 0
                        c[c.notnull()] = 1
                        trainx[n] = c
                        onehotname.append(n)
                    else:
                        dropname.append(n)
                else:
                    nans.append(n)
            
print(len(dropname), len(onehotname), len(nonans), len(ints), len(nans))

331 10 3436 1534 1861


In [17]:
np.sum(trainx[ints].isnull().values)

0

In [18]:
trainx.drop(dropname, axis=1, inplace=True)
train_df.drop(dropname, axis=1, inplace=True)

In [19]:
x_nonans = trainx[nonans]
x_nans = trainx[nans]
# find the sames in x_nonans and x_nans


In [20]:
means = []
stds = []
drop_names = []
names = x_nonans.columns
for n in names:
    c = x_nonans[n]
    tmpm = np.mean(c)
    tmps = np.std(c)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

1430 1431
0


In [21]:
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

534


In [22]:
drop_names = []
for i in sames:
    ci = x_nonans[names[i[0]]]
    for j in i[1:]:
        cj = x_nonans[names[j]]
        sn = np.sum(ci == cj)
        if sn > 700:
            drop_names.append(names[j])
print(len(drop_names))

2005


In [23]:
print(x_nonans.shape)
x_nonans.drop(drop_names, axis=1, inplace=True)
print(x_nonans.shape)

(721, 3436)
(721, 1431)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
means = []
stds = []
drop_names = []
names = x_nans.columns
for n in names:
    c = x_nans[n]
    tmpm = np.mean(c)
    tmps = np.std(c)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

720 719
0


In [25]:
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

166


In [29]:
drop_names = []
names = x_nans.columns
for i in sames:
    ci = x_nans[names[i[0]]]
    for j in i[1:]:
        cj = x_nans[names[j]]
        s1 = np.sum(ci.isnull()==cj.isnull())
        s2 = np.sum(ci[ci.notnull()]==cj[cj.notnull()])
        sn = s1+s2
        if sn > 700:
            drop_names.append(names[j])
print(len(drop_names))

1141


In [31]:
print(x_nans.shape)
x_nans.drop(drop_names, axis=1, inplace=True)
print(x_nans.shape)

(721, 1861)
(721, 720)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [32]:
x_ints = trainx[ints]
x_onehots = trainx[onehotname]

In [33]:
means = []
stds = []
drop_names = []
names = x_ints.columns
for n in names:
    c = x_ints[n]
    tmpm = np.mean(c)
    tmps = np.std(c)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

790 750
607


In [34]:
x_ints.drop(drop_names, axis=1, inplace=True)
print(x_ints.shape)

(721, 927)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
x_onehot = trainx[onehotname]
drop_names = []
for i in range(len(onehotname)):
    ni = onehotname[i]
    ci = x_onehot[ni]
    for j in range(i+1, len(onehotname)):
        nj = onehotname[j]
        cj = x_onehot[nj]
        try:
            x = np.sum(ci==cj)
            if x > 700:
                drop_names.append(nj)
        except:
            pass
print(len(set(drop_names)))

0


In [36]:
x_onehot.drop(drop_names, axis=1, inplace=True)
print(x_onehot.shape)

(721, 10)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
# x_onehot, xints, x_nonans, x_nans are prepared

In [37]:
def dict2list(dic:dict):
    ''' 将字典转化为列表 '''
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

def LastRemainJudge(column):
    unique = set(column)
    uf = {}
    for u in unique:
        uf[u] = 0
    for c in column:
        uf[c] += 1
    sl = sorted(dict2list(uf), key = lambda x:x[1], reverse=True)
    x = sl[0][1]/len(column)
    sl = np.array(sl)
    if x < 0.9:
        #print(sl[:, 1]/len(column))
        return True
    else:
        return False

In [38]:
def continueValues(col):
    al = len(col)
    if al <= 1:
        return False
    ret = True
    for i in range(1, al):
        if col[i] - col[i-1] != 1:
            ret = False
            break
    return ret

dropints = []
remains_float = []
remains_onehot = []
for n in x_ints.columns:
    c = x_ints[n].values
    s = np.sort(list(set(c)))
    if str(s[0])[:4] == '2017' and str(s[-1])[4] == '2017':
        remains_float.append(n)
    elif np.sum(s<0) > 0:
        remains_float.append(n)
    else:
        if continueValues(s):
            if len(s) < 11:
                if LastRemainJudge(c):
                    remains_onehot.append(n)
        else:
            if LastRemainJudge(c):
                remains_float.append(n)
print(len(remains_float), len(remains_onehot))

676 97


In [39]:
# one hot nans and nonans
nan_means = []
nan_stds = []
for n in x_nans.columns:
    c = x_nans[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    nan_means.append(tmpm)
    nan_stds.append(tmps)
    assert tmps > 0
    x_nans[n][c.notnull()] = (x_nans[n][c.notnull()] - tmpm) / tmps
print("nans over")
nonan_means = []
nonan_stds = []
for n in x_nonans.columns:
    c = x_nonans[n]
    tmpm = np.mean(c)
    tmps = np.std(c)
    nonan_means.append(tmpm)
    nonan_stds.append(tmps)
    assert tmps > 0
    x_nonans[n] = (x_nonans[n] - tmpm) / tmps
print("nonans over")

/home/magnusterra/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_with(key, value)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


nans over


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


nonans over


In [40]:
# use KNN to fix null
from sklearn.neighbors import KNeighborsRegressor

In [41]:
tmpxn = x_nonans.values
for n in x_nans.columns:
    c = x_nans[n]
    tmpy = c[c.notnull()]
    tmptrainx = tmpxn[c.notnull().values, :]
    tmptestx = tmpxn[c.isnull().values, :]
    knnr = KNeighborsRegressor()
    knnr.fit(tmptrainx, tmpy)
    x_nans[n][c.isnull()] = knnr.predict(tmptestx)
    print(n)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/magnusterra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


210X213
210X215
220X26
220X27
220X28
220X29
220X30
220X31
220X32
220X54
220X55
220X66
220X67
220X75
220X83
220X91
220X107
220X225
220X227
220X228
220X229
220X230
220X231
220X233
220X235
220X241
220X247
220X248
220X249
220X260
220X262
220X266
220X269
220X270
220X271
220X272
220X273
220X274
220X275
220X277
220X279
220X281
220X282
220X284
220X286
220X288
220X289
220X290
220X291
220X293
220X294
220X295
220X296
220X297
220X299
220X300
220X301
220X302
220X303
220X304
220X305
220X306
220X307
220X308
220X309
220X310
220X311
220X312
220X313
220X315
220X316
220X317
220X318
220X321
220X322
220X323
220X324
220X325
220X326
220X327
220X328
220X329
220X330
220X331
220X332
220X333
220X334
220X335
220X336
220X337
220X338
220X339
220X340
220X341
220X342
220X343
220X344
220X345
220X346
220X347
220X348
220X349
220X350
220X351
220X354
220X356
220X357
220X358
220X359
220X360
220X361
220X362
220X363
220X364
220X365
220X366
220X367
220X368
220X369
220X370
220X371
220X372
220X373
220X374
220X375
220X376
220X37

In [42]:
names = x_nans.columns
for i in range(len(names)):
    n = names[i]
    tmpm = nan_means[i]
    tmps = nan_stds[i]
    x_nans[n] = x_nans[n]*tmps+tmpm
names = x_nonans.columns
for i in range(len(names)):
    n = names[i]
    tmpm = nonan_means[i]
    tmps = nonan_stds[i]
    x_nonans[n] = x_nonans[n]*tmps+tmpm

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [43]:
x_intsfloat = x_ints[remains_float]
x_int2oh = x_ints[remains_onehot]

In [44]:
x_oh = pd.concat([x_onehot, x_int2oh], axis=1)
print(x_oh.shape)

(721, 107)


In [45]:
means = []
stds = []
drop_names = []
names = x_nans.columns
for n in names:
    c = x_nans[n]
    tmpm = np.mean(c)
    tmps = np.std(c)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

719 718
0


In [46]:
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

1


In [47]:
drop_names = []
for i in sames:
    ci = x_nans[names[i[0]]]
    for j in i[1:]:
        cj = x_nans[names[j]]
        sn = np.sum(ci == cj)
        if sn > 700:
            drop_names.append(names[j])
print(len(drop_names))

1


In [48]:
print(x_nans.shape)
x_nans.drop(drop_names, axis=1, inplace=True)
print(x_nans.shape)

(721, 720)
(721, 719)


/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [49]:
x_floats = pd.concat([x_nonans, x_nans], axis=1)
print(x_floats.shape)

(721, 2150)


In [50]:
names = x_oh.columns
x_onehot = pd.get_dummies(x_oh)
print(x_onehot.shape)

(721, 131)


In [79]:
mid = x_floats-np.min(x_floats)+1
x_log1p = np.log1p(mid)
del mid
print(np.sum(x_log1p.isnull().values))

0


In [52]:
x_nif = (x_intsfloat-np.mean(x_intsfloat))/np.std(x_intsfloat)

In [53]:
mid = x_nif-np.min(x_nif)+1
nif_log1p = np.log1p(mid)
del mid
print(np.sum(nif_log1p.isnull().values))

0


In [81]:
nif_log1p = (nif_log1p - np.mean(nif_log1p))/np.std(nif_log1p)

In [55]:
x_log1p = (x_log1p - np.mean(x_log1p))/np.std(x_log1p)

In [56]:
x_df = pd.concat([x_log1p, x_onehot], axis=1)
print(x_df.shape)

(721, 2281)


In [82]:
drop_names = []
for n in x_log1p.columns:
    c = x_log1p[n]
    s1 = np.std(c[:500])
    s2 = np.std(c[500:600])
    s = np.std(c)
    if (s == 0) or (s1 == 0) or (s2 == 0):
        print(n, s1, s2, s)
        drop_names.append(n)

210X225 0.0004998057623666177 0.0 0.0004163436774668393


In [83]:
print(len(drop_names))
print(x_log1p.shape)
x_log1p.drop(drop_names, axis=1, inplace=True)
print(x_log1p.shape)

1
(721, 2150)
(721, 2149)


In [84]:
drop_names = []
ratio = 0.5
for n in x_log1p.columns:
    c = x_log1p[n]
    m1 = np.mean(c[:500])
    m2 = np.mean(c[500:600])
    s1 = np.std(c[:500])
    s2 = np.std(c[500:600])
    r = s2/s1
    if r > 1:
        r = 1/r
    if np.abs(m1-m2)> s1*ratio or r < ratio:
        drop_names.append(n)
print(len(drop_names))

853


In [85]:
# clear data
import copy
xc_log1p = copy.copy(x_log1p)
xc_log1p.drop(drop_names, axis=1, inplace=True)
print(xc_log1p.shape)

(721, 1296)


In [86]:
x_onehot.shape

(721, 131)

In [87]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [88]:
def MSE(y_raw, y_pred):
    print(np.mean(np.square(y_raw-y_pred)))

In [89]:
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV 

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [90]:
from sklearn.model_selection import KFold 

In [91]:
kf = KFold(n_splits=5, random_state=2018)
def get_oof(rgr, xtrain, ytrain, xtest, lenot):
    oof_train = np.zeros((lenot,))
    oof_test = np.zeros((100,))
    oof_test_skf = np.empty((5, 100))
    for i, (train_index, test_index) in enumerate(kf.split(xtrain)):
        kf_xtrain = xtrain[train_index]
        kf_ytrain = ytrain[train_index]
        kf_xtest = xtrain[test_index]
        rgr.fit(kf_xtrain, kf_ytrain)
        oof_train[test_index] = rgr.predict(kf_xtest)
        oof_test_skf[i, : ] = rgr.predict(xtest)
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

In [83]:
xgb = XGBRegressor()
xgb.fit(x_df[:500], train_y)
test1_pred = xgb.predict(x_df[500:600])
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer_Synchronous_logohknn_xgb_20180110.csv', index=False, header=False)

In [92]:
subxtrain, subxval, subytrain, subyval = train_test_split(x_log1p[:500].values, train_y.values, test_size=0.2, random_state=20)


In [95]:
tuned_parameters = {'objective':['reg:linear', 'reg:gamma'], 'max_depth':[1,2,3,4,5,6],
                    'gamma':[0, 1e-3, 1e-2,  1e-4], 'subsample':[0.85, 0.8, 0.75, 0.7],
                   'colsample_bytree':[0.5, 0.75, 1], 'reg_lambda':[1,2]}
scores = ['MSE']
clf = GridSearchCV(estimator =XGBRegressor(), param_grid =tuned_parameters, cv=5, scoring='neg_mean_squared_error')
clf.fit(subxtrain, subytrain)
print("Best parameters set found on development set:")  
print()  
print(clf.best_params_)  
print()  
print("Grid scores on development set:")  
print()  
for params, mean_score, scores in clf.grid_scores_:  
    print("%0.3f (+/-%0.03f) for %r"  
              % (mean_score, scores.std() * 2, params))  
print()
print("The scores are computed on the full evaluation set.")  
print()  
y_true, y_pred = subyval, clf.predict(subxval)
print(MSE(y_true, y_pred))
y_pred = clf.predict(subxtrain)
print(MSE(y_pred, subytrain))

Best parameters set found on development set:

{'colsample_bytree': 0.5, 'gamma': 0.001, 'max_depth': 6, 'objective': 'reg:linear', 'reg_lambda': 2, 'subsample': 0.75}

Grid scores on development set:

-0.038 (+/-0.006) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.85}
-0.039 (+/-0.007) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.8}
-0.038 (+/-0.007) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.75}
-0.039 (+/-0.006) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.7}
-0.038 (+/-0.006) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 2, 'subsample': 0.85}
-0.039 (+/-0.007) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'r

In [96]:
tuned_parameters = {'objective':['reg:linear', 'reg:gamma'], 'max_depth':[1,2,3,4,5,6],
                    'gamma':[0, 1e-3, 1e-2,  1e-4], 'subsample':[0.85, 0.8, 0.75, 0.7],
                   'colsample_bytree':[0.5, 0.75, 1], 'reg_lambda':[1,2]}
scores = ['MSE']
clf = GridSearchCV(estimator =XGBRegressor(), param_grid =tuned_parameters, cv=5, scoring='neg_mean_squared_error')
clf.fit(x_log1p[:500].values, train_y.values)
print("Best parameters set found on development set:")  
print()  
print(clf.best_params_)  
print()  
print("Grid scores on development set:")  
print()  
for params, mean_score, scores in clf.grid_scores_:  
    print("%0.3f (+/-%0.03f) for %r"  
              % (mean_score, scores.std() * 2, params))  
print()
print("The scores are computed on the full evaluation set.")  
print()  
y_true = clf.predict(x_log1p[:500].values)
print(MSE(y_true, train_y.values))

Best parameters set found on development set:

{'colsample_bytree': 0.5, 'gamma': 0.001, 'max_depth': 5, 'objective': 'reg:gamma', 'reg_lambda': 1, 'subsample': 0.75}

Grid scores on development set:

-0.038 (+/-0.014) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.85}
-0.037 (+/-0.014) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.8}
-0.037 (+/-0.013) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.75}
-0.038 (+/-0.013) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.7}
-0.037 (+/-0.014) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 2, 'subsample': 0.85}
-0.037 (+/-0.013) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 're

In [97]:
subxtrain, subxval, subytrain, subyval = train_test_split(xc_log1p[:500].values, train_y.values, test_size=0.2, random_state=20)
tuned_parameters = {'objective':['reg:linear', 'reg:gamma'], 'max_depth':[1,2,3,4,5,6],
                    'gamma':[0, 1e-3, 1e-2,  1e-4], 'subsample':[0.85, 0.8, 0.75, 0.7],
                   'colsample_bytree':[0.5, 0.75, 1], 'reg_lambda':[1,2]}
scores = ['MSE']
clf = GridSearchCV(estimator =XGBRegressor(), param_grid =tuned_parameters, cv=5, scoring='neg_mean_squared_error')
clf.fit(subxtrain, subytrain)
print("Best parameters set found on development set:")  
print()  
print(clf.best_params_)  
print()  
print("Grid scores on development set:")  
print()  
for params, mean_score, scores in clf.grid_scores_:  
    print("%0.3f (+/-%0.03f) for %r"  
              % (mean_score, scores.std() * 2, params))  
print()
print("The scores are computed on the full evaluation set.")  
print()  
y_pred = clf.predict(subxval)
print(MSE(subyval, y_pred))
y_pred = clf.predict(subxtrain)
print(MSE(subytrain, y_pred))

Best parameters set found on development set:

{'colsample_bytree': 0.75, 'gamma': 0.001, 'max_depth': 5, 'objective': 'reg:gamma', 'reg_lambda': 1, 'subsample': 0.85}

Grid scores on development set:

-0.037 (+/-0.005) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.85}
-0.039 (+/-0.005) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.8}
-0.039 (+/-0.005) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.75}
-0.038 (+/-0.006) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.7}
-0.038 (+/-0.005) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 2, 'subsample': 0.85}
-0.039 (+/-0.006) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'r

In [98]:
tuned_parameters = {'objective':['reg:linear', 'reg:gamma'], 'max_depth':[1,2,3,4,5,6],
                    'gamma':[0, 1e-3, 1e-2,  1e-4], 'subsample':[0.85, 0.8, 0.75, 0.7],
                   'colsample_bytree':[0.5, 0.75, 1], 'reg_lambda':[1,2]}
scores = ['MSE']
clf = GridSearchCV(estimator =XGBRegressor(), param_grid =tuned_parameters, cv=5, scoring='neg_mean_squared_error')
clf.fit(xc_log1p[:500].values, train_y.values)
print("Best parameters set found on development set:")  
print()  
print(clf.best_params_)  
print()  
print("Grid scores on development set:")  
print()  
for params, mean_score, scores in clf.grid_scores_:  
    print("%0.3f (+/-%0.03f) for %r"  
              % (mean_score, scores.std() * 2, params))  
print()
print("The scores are computed on the full evaluation set.")  
print()  
y_true = clf.predict(xc_log1p[:500].values)
print(MSE(y_true, train_y.values))

Best parameters set found on development set:

{'colsample_bytree': 0.75, 'gamma': 0.01, 'max_depth': 3, 'objective': 'reg:linear', 'reg_lambda': 2, 'subsample': 0.85}

Grid scores on development set:

-0.038 (+/-0.012) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.85}
-0.037 (+/-0.011) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.8}
-0.037 (+/-0.012) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.75}
-0.037 (+/-0.013) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 1, 'subsample': 0.7}
-0.038 (+/-0.013) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'reg_lambda': 2, 'subsample': 0.85}
-0.037 (+/-0.010) for {'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'r

In [99]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=5, random_state=2018)
def get_oof(rgr, xtrain, ytrain, xtest, lenot):
    oof_train = np.zeros((lenot,))
    oof_test = np.zeros((100,))
    oof_test_skf = np.empty((5, 100))
    for i, (train_index, test_index) in enumerate(kf.split(xtrain)):
        kf_xtrain = xtrain[train_index]
        kf_ytrain = ytrain[train_index]
        kf_xtest = xtrain[test_index]
        rgr.fit(kf_xtrain, kf_ytrain)
        oof_train[test_index] = rgr.predict(kf_xtest)
        oof_test_skf[i, : ] = rgr.predict(xtest)
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

In [103]:
#log1poh xgb = XGBRegressor(gamma=0, max_depth=6, objective='reg:gamma', subsample=0.7)
# log1p
xgb = XGBRegressor(colsample_bytree=0.5, gamma=0.001, max_depth=6, objective='reg:linear', reg_lambda=2, subsample=0.75)
tr_pred, te_pred = get_oof(xgb, x_log1p[:500].values, train_y.values, x_log1p[500:600].values, 500)
print(MSE(tr_pred.reshape(500), train_y))
test1_pred = te_pred.reshape(100)
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer_Synchronous_paramstackknn_log1_xgb_20180111.csv', index=False, header=False)

0.036953953580610883
None


In [104]:
subxtrain, subxval, subytrain, subyval = train_test_split(x_log1p[:500].values, train_y.values, test_size=0.2, random_state=20)

xgb = XGBRegressor(colsample_bytree=0.5, gamma=0.001, max_depth=6, objective='reg:linear', reg_lambda=2, subsample=0.75)
tr_pred, te_pred = get_oof(xgb, subxtrain, subytrain, subxval, 400)
MSE(tr_pred.reshape(400), subytrain)
MSE(te_pred.reshape(100), subyval)


0.0346613107978
0.0251001565079


In [105]:
xgb = XGBRegressor(colsample_bytree=0.75, gamma=0.001, max_depth=5, objective='reg:gamma', reg_lambda=1, subsample=0.85)
tr_pred, te_pred = get_oof(xgb, xc_log1p[:500].values, train_y.values, xc_log1p[500:600].values, 500)
print(MSE(tr_pred.reshape(500), train_y))
test1_pred = te_pred.reshape(100)
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer_Synchronous_paramstackknn_log1reduce_xgb_20180111.csv', index=False, header=False)
subxtrain, subxval, subytrain, subyval = train_test_split(xc_log1p[:500].values, train_y.values, test_size=0.2, random_state=20)

xgb = XGBRegressor(colsample_bytree=0.75, gamma=0.001, max_depth=5, objective='reg:gamma', reg_lambda=1, subsample=0.85)
tr_pred, te_pred = get_oof(xgb, subxtrain, subytrain, subxval, 400)
MSE(tr_pred.reshape(400), subytrain)
MSE(te_pred.reshape(100), subyval)


0.03710315207154316
None
0.0348272236606
0.0245863464438
